Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [18]:
from random import random
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue, SimpleQueue, LifoQueue
import time
%pip install tqdm
from tqdm.auto import tqdm

import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [19]:
PROBLEM_SIZE = 10 # 400
NUM_SETS = 40 # 150
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])
#SETS

In [20]:
def get_n_overlaps(state):
    return np.sum(np.sum([SETS[i] for i in state.taken], axis=0) > 1)

In [21]:
# cheks how many True value are covered in a state
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

# returns how many False need to be covered yet
def distance(state):
    return PROBLEM_SIZE - sum(covered(state))


# --------------------- A* search ---------------------

def g(state):
    return len(state.taken)

def h(state):
    pass

def distance_A_star(state):

    w1 = 2 # distance
    w2 = 0     # cost
    w3 = 1     # overlap
    w_tot = w1 + w2 + w3
     
    # Checks how far are we from teh goal -> lower dist is better
    dist = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    
    # checks how many True value exists in state.taken -> lower cost is better
    cost = PROBLEM_SIZE - dist
    
    # optimization: try to minimize overlap
    n_overlap = get_n_overlaps(state)
    
    return (w1/w_tot) * dist + (w2/w_tot) * cost + (w3/w_tot) * n_overlap 

In [22]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [23]:
def search():

    frontier = deque()
    state = State(set(), set(range(NUM_SETS)))
    frontier.append(state)

    counter = 0
    current_state = frontier.popleft()
    with tqdm(total=None) as pbar:
        while not goal_check(current_state):
            counter += 1
            for action in current_state[1]:
                new_state = State(
                    current_state.taken ^ {action},
                    current_state.not_taken ^ {action},
                )
                frontier.append(new_state)
            current_state = frontier.popleft()
            pbar.update(1)

    return counter, current_state

def search_A_star():

    frontier = PriorityQueue()  
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((distance_A_star(state), state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        # print(current_state)
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((distance_A_star(new_state), new_state))
            # print(f'taken: {new_state.taken}')
            # print(f'Not taken: {new_state.not_taken}')
            # print()
        _, current_state = frontier.get()
    
    return counter, current_state

In [24]:
# set distance function:    distance   distance_A_star
start_t1 = time.time()
print('Solving Search 1 (Breadth first) ...')
counter1, current_state1 = search()
end_t1 = time.time()
print(f"Solved in {counter1:,} steps ({len(current_state1.taken)} tiles)")
print(f'# overlaps: {get_n_overlaps(current_state1)}')
print(f'(time: {end_t1-start_t1:.2f} s)')

print()

start_t2 = time.time()
print('Solving Search 2 (A* search) ...')
counter2, current_state2 = search_A_star()
end_t2 = time.time()
print(f"Solved in {counter2:,} steps ({len(current_state2.taken)} tiles)")
print(f'# overlaps: {get_n_overlaps(current_state2)}')
print(f'(time: {end_t2-start_t2:.2f} s)')

Solving Search 1 (Breadth first) ...


3154it [00:01, 2046.69it/s]


Solved in 3,154 steps (3 tiles)
# overlaps: 1
(time: 1.59 s)

Solving Search 2 (A* search) ...
Solved in 3 steps (3 tiles)
# overlaps: 2
(time: 0.01 s)


In [25]:
is_print_solutions = True # False

if is_print_solutions:
    print(f'SOLUTIONS:\n')
    print('Original search:')
    for s in current_state1.taken:
        print(f'{SETS[s]}\t<- {s}')
    print()
    print('A* search:')
    for s in current_state2.taken:
        print(f'{SETS[s]}\t<- {s}')

SOLUTIONS:

Original search:
[False  True  True False  True False False False  True False]	<- 1
[False False False False False False False  True False  True]	<- 2
[ True  True False  True False  True  True False False False]	<- 35

A* search:
[ True  True False  True False  True  True False False False]	<- 35
[False False False False  True False False False False  True]	<- 3
[ True False  True False False False False  True  True  True]	<- 7


# Compare 2 algorithms

In [2515]:
def score(n_steps, n_tiles, n_overlap, t):
    return 1/(n_steps*n_tiles*n_overlap*t)

In [ ]:
SIM_ITER = 1000

a1 = 'Original'
a2 = 'A*'

scores_a1 = []
scores_a2 = []

for _ in range(SIM_ITER):

    SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
    
    start_t1 = time.time()
    counter1, current_state1 = search()
    end_t1 = time.time()
    score1 = score(counter1, len(current_state1.taken), get_n_overlaps(current_state1), end_t1-start_t1)
    scores_a1.append(score1)

    start_t2 = time.time()
    counter2, current_state2 = search()
    end_t2 = time.time()
    score2 = score(counter2, len(current_state2.taken), get_n_overlaps(current_state2), end_t2-start_t2)
    scores_a2.append(score2)

print(f'SCORES:\n')
print(f'Score algorithm 1 ({a1}): {sum(scores_a1)/SIM_ITER:.4f}')
print(f'Score algorithm 2 ({a2}): {sum(scores_a2)/SIM_ITER:.4f}')